# View the Data file : Data

In [1]:
import pandas as pd
from geopy.distance import geodesic
import sys
from heapq import heappop, heappush
import folium
import numpy as np
df2 = pd.read_csv('bus_stop_graph.csv')
df2.head(100)

,Source,Destination,Distance
0,1,2,53.98
1,2,3,50.07
2,3,4,36.87
3,4,5,74.76
4,5,6,55.36
5,6,1,271.04
6,7,8,53.98
7,8,9,50.07
8,9,10,36.87
9,10,11,74.76


In [2]:
import pandas as pd
from geopy.distance import geodesic
import sys
from heapq import heappop, heappush
import folium
import numpy as np
bus_stop_coordinates = pd.read_csv('bus_stop_coordinates.csv')
bus_stop_coordinates.head(100)

,stop_id,latitude,longitude
0,1,11.589511,104.918883
1,2,12.053914,105.063490
2,3,12.500175,105.124814
3,4,12.753342,104.905378
4,5,13.128389,104.332836
5,6,13.359122,103.879603
6,7,11.577754,104.919609
7,8,12.003479,104.950829
8,9,12.550849,105.080591
9,10,13.285133,104.066074


# Nearest bus stops:

In [3]:
def find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops=1):
    distances = []
    for index, row in bus_stop_coordinates.iterrows():
        # Store stop_id and coordinates
        stop = row['stop_id']
        coords = (row['latitude'], row['longitude'])
        
        # Skip if coordinates are NaN
        if pd.isnull(coords[0]) or pd.isnull(coords[1]):
            continue  # Skip this bus stop and move to the next

        # Calculate the distance and add to list
        distance = geodesic(user_location, coords).meters
        distances.append((stop, distance))

    # Sort by distance and return the closest stops
    distances.sort(key=lambda x: x[1])  
    return distances[:num_stops]

user_location = (11.5822, 104.91568) 

nearest_stops = find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops=2)

# Display the nearest bus stops
print("Nearest bus stops:")
for stop, distance in nearest_stops:
    print(f"Stop ID {stop}: {distance:.2f} meters")

# Create a map centered around the user's location
my_map = folium.Map(location=user_location, zoom_start=13)

# Add a marker for the user's location
folium.Marker(
    location=user_location, 
    popup="Your Location", 
    icon=folium.Icon(color='blue')
).add_to(my_map)

# Add markers for the nearest bus stops
for stop, distance in nearest_stops:
    # Retrieve the coordinates from the DataFrame for the specific stop
    coords = bus_stop_coordinates.loc[bus_stop_coordinates['stop_id'] == stop, ['latitude', 'longitude']].values[0]
    folium.Marker(
        location=coords, 
        popup=f"Stop ID {stop} ({distance:.2f} meters)",
        icon=folium.Icon(color='orange', prefix='fa', icon='bus')
    ).add_to(my_map)

# Display the map
my_map

Nearest bus stops:
Stop ID 7.0: 652.36 meters
Stop ID 1.0: 880.94 meters


In [4]:
# prompt: when we stay other location and then we fine bus station that shorter distance from we stay in by pin location on map  to all bus , on map, we can find location on the map, on map, show on map, three choices for bus station

import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
def find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops=1):
    distances = []
    for _, row in bus_stop_coordinates.iterrows():
        stop = row['stop_id']
        coords = (row['latitude'], row['longitude'])
        
        # Skip if coordinates are NaN
        if pd.isnull(coords[0]) or pd.isnull(coords[1]):
            continue  # Skip this bus stop and move to the next
        
        distance = geodesic(user_location, coords).meters
        distances.append((stop, distance))

    distances.sort(key=lambda x: x[1])  # Sort by distance
    return distances[:num_stops]

# Example user location
user_location = (11.5822, 104.91568)  # Replace with actual user location

# Find the nearest bus stops
nearest_stops = find_nearest_bus_stops(user_location, bus_stop_coordinates, 2)

# Display the nearest bus stops
print("Nearest bus stops:")
for stop, distance in nearest_stops:
    print(f"Stop ID {stop}: {distance:.2f} meters")

# Create a map centered around the user's location
my_map = folium.Map(location=user_location, zoom_start=13)

# Add a marker for the user's location
folium.Marker(
    location=user_location, 
    popup="Your Location", 
    icon=folium.Icon(color='blue')
).add_to(my_map)

# Add markers for the nearest bus stops
for stop, distance in nearest_stops:
    # Retrieve the coordinates from the DataFrame for the specific stop
    coords = bus_stop_coordinates.loc[bus_stop_coordinates['stop_id'] == stop, ['latitude', 'longitude']].values[0]
    folium.Marker(
        location=coords, 
        popup=f"Stop ID {stop} ({distance:.2f} meters)",
        icon=folium.Icon(color='orange', prefix='fa', icon='bus')
    ).add_to(my_map)

# Display the map
my_map


Nearest bus stops:
Stop ID 7.0: 652.36 meters
Stop ID 1.0: 880.94 meters


# Bus Tracking 01 🚏

In [5]:
import pandas as pd
import sys
from heapq import heappop, heappush
import folium
import numpy as np

# Define the Dijkstra's algorithm function
def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    # Initialize node data with float keys
    node_data = {node: {'cost': inf, 'pred': None} for node in graph}
    node_data[src]['cost'] = 0
    visited = set()
    heap = [(0, src)]

    while heap:
        cost, node = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph.get(node, {}).items():
            if neighbor not in node_data:
                node_data[neighbor] = {'cost': inf, 'pred': None}
            new_cost = cost + weight
            if new_cost < node_data[neighbor]['cost']:
                node_data[neighbor]['cost'] = new_cost
                node_data[neighbor]['pred'] = node
                heappush(heap, (new_cost, neighbor))

    # Reconstruct path from destination to source
    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
    print("Shortest Distance:", shortest_distance)
    print("Shortest Path:", path)

    return shortest_distance, path

# Load the bus stop graph from CSV into a DataFrame
bus_stop_graph_df = pd.read_csv('bus_stop_graph.csv')

# Initialize the graph dictionary from the DataFrame
bus_stop_graph = {}
for _, row in bus_stop_graph_df.iterrows():
    start = float(row['Source'])
    end = float(row['Destination'])
    distance = row['Distance']
    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance

# Example source and destination stops (as floats)
source_stop = 1.0
destination_stop = 6.0

# Run Dijkstra's algorithm
shortest_distance, path = dijkstra(bus_stop_graph, source_stop, destination_stop)

Shortest Distance: 271.04
Shortest Path: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]


In [6]:
import pandas as pd
import sys
from heapq import heappop, heappush
import folium
import numpy as np

def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    # Initialize node_data with appropriate keys for Dijkstra's algorithm
    node_data = {node: {'cost': inf, 'pred': None} for node in graph}
    node_data[src]['cost'] = 0
    visited = set()
    heap = [(0, src)]

    while heap:
        cost, node = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph[node].items():
            if neighbor in node_data:
                new_cost = cost + weight
                if new_cost < node_data[neighbor]['cost']:
                    node_data[neighbor]['cost'] = new_cost
                    node_data[neighbor]['pred'] = node
                    heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
    print("Shortest Distance:", shortest_distance)
    print("Shortest Path:", path)

    return shortest_distance, path

def calculate_bus_cost(distance):
    # Example pricing model for cost calculation
    base_cost = 1500  # Base cost in local currency (e.g., Riel)
    distance_factor = 0.00  # Cost per meter
    cost = base_cost + (distance * distance_factor)
    return cost

# Load graph and coordinates from CSV files
bus_stop_graph_df = pd.read_csv('bus_stop_graph.csv')
bus_stop_coordinates_df = pd.read_csv('bus_stop_coordinates.csv')

# Convert bus_stop_graph_df into a dictionary format suitable for Dijkstra's function
bus_stop_graph = {}
for _, row in bus_stop_graph_df.iterrows():
    start = float(row['Source'])
    end = float(row['Destination'])
    distance = row['Distance']
    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance

# Convert bus_stop_coordinates_df into a dictionary of coordinates
bus_stop_coordinates = {}
for _, row in bus_stop_coordinates_df.iterrows():
    stop = float(row['stop_id'])
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)

# Example source and destination stops
source_stop = 1.0
destination_stop = 6.0

# Run Dijkstra's algorithm
shortest_distance, path = dijkstra(bus_stop_graph, source_stop, destination_stop)

# Create a map centered around the source stop
map_center = bus_stop_coordinates[source_stop]
my_map = folium.Map(location=map_center, zoom_start=13)

# Add markers for start and destination only
folium.Marker(location=bus_stop_coordinates[source_stop], popup="Start: " + str(source_stop),
              icon=folium.Icon(color='green', prefix='fa', icon='bus')).add_to(my_map)

folium.Marker(location=bus_stop_coordinates[destination_stop], popup="Destination: " + str(destination_stop),
              icon=folium.Icon(color='red', prefix='fa', icon='bus')).add_to(my_map)

# Initialize total distance, total cost, and directions list
total_distance = 0
total_cost = 0
directions = []

# Iterate over path to calculate segments and display directions with appropriate icons and line styles
for i in range(1, len(path)):
    start = path[i - 1]
    end = path[i]
    distance = bus_stop_graph[start][end]
    total_distance += distance
    cost = calculate_bus_cost(distance)
    total_cost += cost
    
    # Choose transportation mode and line style
    if distance <= 500:
        mode = "Walk"
        line_style = {'color': 'blue', 'weight': 2.5, 'dash_array': '5, 5'}  # Dashed line for walking
    else:
        mode = "Drive"
        line_style = {'color': 'red', 'weight': 2.5, 'opacity': 1}  # Solid line for driving

    # Draw the path segment with the chosen style
    folium.PolyLine([bus_stop_coordinates[start], bus_stop_coordinates[end]], **line_style).add_to(my_map)
    
    # Add direction information
    directions.append(f"{mode} from {start} to {end}. Distance: {distance} meters. Cost: {cost:.2f} Riel.")

# Display directions and total values
print("\nDirections (Walking/Car):")
for direction in directions:
    print(direction)
print(f"Total Distance: {total_distance} meters")
print(f"Total Cost: {total_cost:.2f} Riel")

# Display the map
my_map


Shortest Distance: 271.04
Shortest Path: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

Directions (Walking/Car):
Walk from 1.0 to 2.0. Distance: 53.98 meters. Cost: 1500.00 Riel.
Walk from 2.0 to 3.0. Distance: 50.07 meters. Cost: 1500.00 Riel.
Walk from 3.0 to 4.0. Distance: 36.87 meters. Cost: 1500.00 Riel.
Walk from 4.0 to 5.0. Distance: 74.76 meters. Cost: 1500.00 Riel.
Walk from 5.0 to 6.0. Distance: 55.36 meters. Cost: 1500.00 Riel.
Total Distance: 271.04 meters
Total Cost: 7500.00 Riel


In [7]:
# # prompt: show icon start and destination only

# import pandas as pd
# import sys
# from heapq import heappop, heappush
# import folium
# import numpy as np

# def dijkstra(graph, src, dest, walking_threshold=200):
#     inf = sys.maxsize
#     # Ensure node_data keys are of the same type as graph keys
#     node_data = {node: {'cost': inf, 'pred': None} for node in graph}
#     node_data[src]['cost'] = 0
#     visited = set()
#     heap = [(0, src)]

#     while heap:
#         (cost, node) = heappop(heap)
#         if node in visited:
#             continue
#         visited.add(node)

#         for neighbor, weight in graph[node].items():
#             # Ensure neighbor is in node_data before accessing it
#             if neighbor in node_data:
#                 new_cost = cost + weight
#                 if new_cost < node_data[neighbor]['cost']:
#                     node_data[neighbor]['cost'] = new_cost
#                     node_data[neighbor]['pred'] = node
#                     heappush(heap, (new_cost, neighbor))

#     def reconstruct_path(node):
#         path = []
#         while node is not None:
#             path.append(node)
#             node = node_data[node]['pred']
#         return path[::-1]

#     shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
#     print("Shortest Distance:", shortest_distance)
#     print("Shortest Path:", path)

#     return shortest_distance, path

# def calculate_bus_cost(distance):
#     # Example pricing model:
#     base_cost = 1500  # Base cost for local buses
#     distance_factor = 0.00  # Cost per meter
#     cost = base_cost + (distance * distance_factor)
#     return cost

# bus_stop_graph = pd.read_csv('bus_stop_graph.csv')

# bus_stop_graph = {}
# for index, row in bus_stop_graph.iterrows():
#     start = row['Source']
#     end = row['Destination']
#     distance = row['Distance']

#   # This code needs to be inside the loop
#     if start not in bus_stop_graph:
#         bus_stop_graph[start] = {}
#     bus_stop_graph[start][end] = distance

# # Assuming the CSV file is named 'data.csv'
# bus_stop_coordinates = pd.read_csv('bus_stop_coordinates.csv')
# # Extract bus stop coordinates from the CSV file
# bus_stop_coordinates = {}
# for index, row in bus_stop_coordinates.iterrows():
#     stop = row['stop_id']
#     lat = row['latitude']
#     lon = row['longitude']
#     bus_stop_coordinates[stop] = (lat, lon)


# # Indent the following lines to be inside the main body of the script
# source_stop = 1.0  # Change to float to match keys in bus_stop_graph
# destination_stop = 6.0  # Change to float
# shortest_distance, path = dijkstra(bus_stop_graph, source_stop, destination_stop)

# # Create a map centered around the first bus stop
# map_center = bus_stop_coordinates[source_stop]
# my_map = folium.Map(location=map_center, zoom_start=13)


# # Add markers for start and destination only
# folium.Marker(location=bus_stop_coordinates[source_stop], popup=source_stop,
#               icon=folium.Icon(color='green', prefix='fa', icon='bus')).add_to(my_map)

# folium.Marker(location=bus_stop_coordinates[destination_stop], popup=destination_stop,
#               icon=folium.Icon(color='red', prefix='fa', icon='bus')).add_to(my_map)

#  # Enhanced directions with transportation mode (Walking or Car) and line styles
# total_distance = 0
# directions = []
# costs = []  # Create a list to store costs for each segment
# for i in range(1, len(path)):
#     start = path[i - 1]
#     end = path[i]
#     #Ensure that start and end are the correct datatype for the bus_stop_graph dictionary
#     start = int(start) if start.is_integer() else start
#     end = int(end) if end.is_integer() else end
#     distance = bus_stop_graph[start][end]
#     total_distance += distance
#     # Calculate cost based on distance
#     cost = calculate_bus_cost(distance)
#     costs.append(cost)  # Append the cost to the list
#     # Choose transportation mode based on distance (example threshold)
#     if distance <= 500:
#         mode = "Walk"
#         line_style = {'color': 'blue', 'weight': 2.5, 'dash_array': '5, 5'}  # Dashed line for walking
#     else:
#         mode = "Drive"
#         line_style = {'color': 'red', 'weight': 2.5, 'opacity': 1}  # Solid line for driving

#     # Draw the segment of the path with appropriate style
#     folium.PolyLine([bus_stop_coordinates[start], bus_stop_coordinates[end]],
#                     **line_style).add_to(my_map)

#     directions.append(f"{mode} from {start} to {end}. Distance: {distance} meters. Cost: {cost} USD.")
# print("\nDirections (Walking/Car):")
# total_cost = 0  # Initialize total cost
# for i in range(1, len(path)):
#     start = path[i - 1]
#     end = path[i]
#     distance = bus_stop_graph[start][end]
#     total_distance += distance
#     cost = calculate_bus_cost(distance)  # Calculate cost for this segment
#     total_cost += cost  # Accumulate total cost
# print(f"Total Distance: {total_distance} meters")
# print(f"Total Cost: {sum(costs)} Riel")  # Calculate and print the total cost
# # Display the map
# my_map

In [8]:
# prompt: bus tracking

import pandas as pd
from geopy.distance import geodesic
import sys
from heapq import heappop, heappush
import folium
import numpy as np

# Load bus stop coordinates and graph data
df2 = pd.read_csv('bus_stop_coordinates.csv')
df3 = pd.read_csv('bus_stop_graph.csv')

# Function to calculate distance between two bus stops
def calculate_distance(from_stop_id, to_stop_id, df2):
  """Calculates the geodesic distance between two bus stops."""

  from_stop_coords = df2[df2['stop_id'] == from_stop_id][['latitude', 'longitude']].values
  to_stop_coords = df2[df2['stop_id'] == to_stop_id][['latitude', 'longitude']].values

  if from_stop_coords.size > 0 and to_stop_coords.size > 0:
      return geodesic(from_stop_coords[0], to_stop_coords[0]).km
  else:
      return None


# Create a graph representation of bus routes
bus_stop_graph = {}
for index, row in df3.iterrows():
    start = row['Source']
    end = row['Destination']
    distance = row['Distance']

    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance


# Create a dictionary of bus stop coordinates
bus_stop_coordinates = {}
for index, row in df2.iterrows():
    stop = row['stop_id']
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)


# Dijkstra's algorithm for finding shortest path
def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    node_data = {node: {'cost': inf, 'pred': None} for node in graph}
    node_data[src]['cost'] = 0
    visited = set()
    heap = [(0, src)]

    while heap:
        (cost, node) = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph[node].items():
            if neighbor in node_data:
                new_cost = cost + weight
                if new_cost < node_data[neighbor]['cost']:
                    node_data[neighbor]['cost'] = new_cost
                    node_data[neighbor]['pred'] = node
                    heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
    return shortest_distance, path


# Function to calculate bus cost based on distance
def calculate_bus_cost(distance):
    # Example pricing model:
    base_cost = 1500  # Base cost for local buses
    distance_factor = 0.00  # Cost per meter
    cost = base_cost + (distance * distance_factor)
    return cost


# Function to find the nearest bus stops to a given location
def find_nearest_bus_stops(location, bus_stop_coordinates, num_stops=1):
    distances = []
    for stop_id, coords in bus_stop_coordinates.items():
        distance = geodesic(location, coords).km
        distances.append((stop_id, distance))

    distances.sort(key=lambda item: item[1])
    return distances[:num_stops]


# Function to find nearest stops for a route
def find_nearest_stops_for_route(user_location, destination, bus_stop_coordinates, num_stops=1):
    nearest_to_user = find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops)
    nearest_to_destination = find_nearest_bus_stops(destination, bus_stop_coordinates, num_stops)
    return nearest_to_user, nearest_to_destination
# Function to get route options
def get_route_options(user_location, destination, nearest_to_user, nearest_to_destination, bus_stop_graph):
    route_options = []
    for start_stop, _ in nearest_to_user:
        for end_stop, _ in nearest_to_destination:
            try:
                bus_distance, path = dijkstra(bus_stop_graph, float(start_stop), float(end_stop))
                walking_distance_start = geodesic(user_location, bus_stop_coordinates[start_stop]).meters
                walking_distance_end = geodesic(destination, bus_stop_coordinates[end_stop]).meters
                total_distance = bus_distance + walking_distance_start + walking_distance_end
                route_options.append({
                    'start_stop': start_stop,
                    'end_stop': end_stop,
                    'total_distance': total_distance,
                    'path': path
                })
            except KeyError:
                pass  # Handle cases where there's no bus route between the stops

    route_options.sort(key=lambda x: x['total_distance'])
    return route_options[:1]  # Return top 3 options


# Function to display route options on map
def display_route_options_on_map(user_location, destination, route_options, bus_stop_coordinates):
    my_map = folium.Map(location=[(user_location[0] + destination[0]) / 2, (user_location[1] + destination[1]) / 2], zoom_start=13)

    folium.Marker(location=user_location, popup="Your Location", icon=folium.Icon(color='blue')).add_to(my_map)
    folium.Marker(location=destination, popup="Destination", icon=folium.Icon(color='red')).add_to(my_map)

    colors = ['blue', 'orange', 'purple']
    for i, option in enumerate(route_options):
        folium.PolyLine([user_location, bus_stop_coordinates[option['start_stop']]],
                         color='gray', weight=2.5, opacity=1, dash_array='5',
                         tooltip=f"Walk to {option['start_stop']}").add_to(my_map)

        bus_route_coords = [bus_stop_coordinates[stop] for stop in option['path']]
        folium.PolyLine(bus_route_coords, color=colors[i], weight=3.5, opacity=1,
                         tooltip=f"Bus Route: {option['start_stop']} to {option['end_stop']}").add_to(my_map)

        folium.PolyLine([bus_stop_coordinates[option['end_stop']], destination],
                         color='gray', weight=2.5, opacity=1, dash_array='5',
                         tooltip=f"Walk to Destination").add_to(my_map)

    return my_map


# Example usage:
user_location = (11.54959056969515, 104.91341843256426)
destination = (13.290265029330493, 104.0616138735088)

nearest_to_user, nearest_to_destination = find_nearest_stops_for_route(
    user_location, destination, bus_stop_coordinates
)

route_options = get_route_options(
    user_location, destination, nearest_to_user, nearest_to_destination, bus_stop_graph
)

route_map = display_route_options_on_map(user_location, destination, route_options, bus_stop_coordinates)
route_map

# Bus Tracking 02

In [9]:
# prompt: show icon start and destination only

import pandas as pd
import sys
from heapq import heappop, heappush
import folium
import numpy as np

def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    # Ensure node_data keys are of the same type as graph keys
    node_data = {node: {'cost': inf, 'pred': None} for node in graph}
    node_data[src]['cost'] = 0
    visited = set()
    heap = [(0, src)]

    while heap:
        (cost, node) = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph[node].items():
            # Ensure neighbor is in node_data before accessing it
            if neighbor in node_data:
                new_cost = cost + weight
                if new_cost < node_data[neighbor]['cost']:
                    node_data[neighbor]['cost'] = new_cost
                    node_data[neighbor]['pred'] = node
                    heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
    print("Shortest Distance:", shortest_distance)
    print("Shortest Path:", path)

    return shortest_distance, path

def calculate_bus_cost(distance):
    # Example pricing model:
    base_cost = 1500  # Base cost for local buses
    distance_factor = 0.00  # Cost per meter
    cost = base_cost + (distance * distance_factor)
    return cost

df3 = pd.read_csv('bus_stop_graph.csv')

bus_stop_graph = {}
for index, row in df3.iterrows():
    start = row['Source']
    end = row['Destination']
    distance = row['Distance']

  # This code needs to be inside the loop
    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance

# Assuming the CSV file is named 'data.csv'
df2 = pd.read_csv('bus_stop_coordinates.csv')
# Extract bus stop coordinates from the CSV file
bus_stop_coordinates = {}
for index, row in df2.iterrows():
    stop = row['stop_id']
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)


# Indent the following lines to be inside the main body of the script
source_stop = 1.0  # Change to float to match keys in bus_stop_graph
destination_stop = 6.0  # Change to float
shortest_distance, path = dijkstra(bus_stop_graph, source_stop, destination_stop)

# Create a map centered around the first bus stop
map_center = bus_stop_coordinates[source_stop]
my_map = folium.Map(location=map_center, zoom_start=13)


# Add markers for start and destination only
folium.Marker(location=bus_stop_coordinates[source_stop], popup=source_stop,
              icon=folium.Icon(color='green', prefix='fa', icon='bus')).add_to(my_map)

folium.Marker(location=bus_stop_coordinates[destination_stop], popup=destination_stop,
              icon=folium.Icon(color='red', prefix='fa', icon='bus')).add_to(my_map)

 # Enhanced directions with transportation mode (Walking or Car) and line styles
total_distance = 0
directions = []
costs = []  # Create a list to store costs for each segment
for i in range(1, len(path)):
    start = path[i - 1]
    end = path[i]
    #Ensure that start and end are the correct datatype for the bus_stop_graph dictionary
    start = int(start) if start.is_integer() else start
    end = int(end) if end.is_integer() else end
    distance = bus_stop_graph[start][end]
    total_distance += distance
    # Calculate cost based on distance
    cost = calculate_bus_cost(distance)
    costs.append(cost)  # Append the cost to the list
    # Choose transportation mode based on distance (example threshold)
    if distance <= 500:
        mode = "Walk"
        line_style = {'color': 'blue', 'weight': 2.5, 'dash_array': '5, 5'}  # Dashed line for walking
    else:
        mode = "Drive"
        line_style = {'color': 'red', 'weight': 2.5, 'opacity': 1}  # Solid line for driving

    # Draw the segment of the path with appropriate style
    folium.PolyLine([bus_stop_coordinates[start], bus_stop_coordinates[end]],
                    **line_style).add_to(my_map)

    directions.append(f"{mode} from {start} to {end}. Distance: {distance} meters. Cost: {cost} USD.")
print("\nDirections (Walking/Car):")
total_cost = 0  # Initialize total cost
for i in range(1, len(path)):
    start = path[i - 1]
    end = path[i]
    distance = bus_stop_graph[start][end]
    total_distance += distance
    cost = calculate_bus_cost(distance)  # Calculate cost for this segment
    total_cost += cost  # Accumulate total cost
print(f"Total Distance: {total_distance} meters")
print(f"Total Cost: {sum(costs)} Riel")  # Calculate and print the total cost
# Display the map
my_map

Shortest Distance: 271.04
Shortest Path: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

Directions (Walking/Car):
Total Distance: 542.08 meters
Total Cost: 7500.0 Riel


In [10]:
# prompt: bus tracking
import pandas as pd
from geopy.distance import geodesic
import sys
from heapq import heappop, heappush
import folium
import numpy as np

# Load bus stop coordinates and graph data
df2 = pd.read_csv('bus_stop_coordinates.csv')
df3 = pd.read_csv('bus_stop_graph.csv')

# Function to calculate distance between two bus stops
def calculate_distance(from_stop_id, to_stop_id, df2):
  """Calculates the geodesic distance between two bus stops."""

  from_stop_coords = df2[df2['stop_id'] == from_stop_id][['latitude', 'longitude']].values
  to_stop_coords = df2[df2['stop_id'] == to_stop_id][['latitude', 'longitude']].values

  if from_stop_coords.size > 0 and to_stop_coords.size > 0:
      return geodesic(from_stop_coords[0], to_stop_coords[0]).km
  else:
      return None


# Create a graph representation of bus routes
bus_stop_graph = {}
for index, row in df3.iterrows():
    start = row['Source']
    end = row['Destination']
    distance = row['Distance']

    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance


# Create a dictionary of bus stop coordinates
bus_stop_coordinates = {}
for index, row in df2.iterrows():
    stop = row['stop_id']
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)


# Dijkstra's algorithm for finding shortest path
def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    node_data = {node: {'cost': inf, 'pred': None} for node in graph}
    node_data[src]['cost'] = 0
    visited = set()
    heap = [(0, src)]

    while heap:
        (cost, node) = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph[node].items():
            if neighbor in node_data:
                new_cost = cost + weight
                if new_cost < node_data[neighbor]['cost']:
                    node_data[neighbor]['cost'] = new_cost
                    node_data[neighbor]['pred'] = node
                    heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
    return shortest_distance, path


# Function to calculate bus cost based on distance
def calculate_bus_cost(distance):
    # Example pricing model:
    base_cost = 1500  # Base cost for local buses
    distance_factor = 0.00  # Cost per meter
    cost = base_cost + (distance * distance_factor)
    return cost


# Function to find the nearest bus stops to a given location
def find_nearest_bus_stops(location, bus_stop_coordinates, num_stops=1):
    distances = []
    for stop_id, coords in bus_stop_coordinates.items():
        distance = geodesic(location, coords).km
        distances.append((stop_id, distance))

    distances.sort(key=lambda item: item[1])
    return distances[:num_stops]


# Function to find nearest stops for a route
def find_nearest_stops_for_route(user_location, destination, bus_stop_coordinates, num_stops=1):
    nearest_to_user = find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops)
    nearest_to_destination = find_nearest_bus_stops(destination, bus_stop_coordinates, num_stops)
    return nearest_to_user, nearest_to_destination
# Function to get route options
def get_route_options(user_location, destination, nearest_to_user, nearest_to_destination, bus_stop_graph):
    route_options = []
    for start_stop, _ in nearest_to_user:
        for end_stop, _ in nearest_to_destination:
            try:
                bus_distance, path = dijkstra(bus_stop_graph, float(start_stop), float(end_stop))
                walking_distance_start = geodesic(user_location, bus_stop_coordinates[start_stop]).meters
                walking_distance_end = geodesic(destination, bus_stop_coordinates[end_stop]).meters
                total_distance = bus_distance + walking_distance_start + walking_distance_end
                route_options.append({
                    'start_stop': start_stop,
                    'end_stop': end_stop,
                    'total_distance': total_distance,
                    'path': path
                })
            except KeyError:
                pass  # Handle cases where there's no bus route between the stops

    route_options.sort(key=lambda x: x['total_distance'])
    return route_options[:1]  # Return top 3 options


# Function to display route options on map
def display_route_options_on_map(user_location, destination, route_options, bus_stop_coordinates):
    my_map = folium.Map(location=[(user_location[0] + destination[0]) / 2, (user_location[1] + destination[1]) / 2], zoom_start=13)

    folium.Marker(location=user_location, popup="Your Location", icon=folium.Icon(color='blue')).add_to(my_map)
    folium.Marker(location=destination, popup="Destination", icon=folium.Icon(color='red')).add_to(my_map)

    colors = ['blue', 'orange', 'purple']
    for i, option in enumerate(route_options):
        folium.PolyLine([user_location, bus_stop_coordinates[option['start_stop']]],
                         color='gray', weight=2.5, opacity=1, dash_array='5',
                         tooltip=f"Walk to {option['start_stop']}").add_to(my_map)

        bus_route_coords = [bus_stop_coordinates[stop] for stop in option['path']]
        folium.PolyLine(bus_route_coords, color=colors[i], weight=3.5, opacity=1,
                         tooltip=f"Bus Route: {option['start_stop']} to {option['end_stop']}").add_to(my_map)

        folium.PolyLine([bus_stop_coordinates[option['end_stop']], destination],
                         color='gray', weight=2.5, opacity=1, dash_array='5',
                         tooltip=f"Walk to Destination").add_to(my_map)

    return my_map


# Example usage:
user_location = (11.554245567246813, 104.90754219475156)
destination = (13.284128007937076, 104.06482052949444)

nearest_to_user, nearest_to_destination = find_nearest_stops_for_route(
    user_location, destination, bus_stop_coordinates
)

route_options = get_route_options(
    user_location, destination, nearest_to_user, nearest_to_destination, bus_stop_graph
)

route_map = display_route_options_on_map(user_location, destination, route_options, bus_stop_coordinates)
route_map

In [11]:
# prompt: bus tracking
import pandas as pd
from geopy.distance import geodesic
import sys
from heapq import heappop, heappush
import folium
import numpy as np

# Load bus stop coordinates and graph data
df2 = pd.read_csv('bus_stop_coordinates.csv')
df3 = pd.read_csv('bus_stop_graph.csv')

# Function to calculate distance between two bus stops
def calculate_distance(from_stop_id, to_stop_id, df2):
  """Calculates the geodesic distance between two bus stops."""

  from_stop_coords = df2[df2['stop_id'] == from_stop_id][['latitude', 'longitude']].values
  to_stop_coords = df2[df2['stop_id'] == to_stop_id][['latitude', 'longitude']].values

  if from_stop_coords.size > 0 and to_stop_coords.size > 0:
      return geodesic(from_stop_coords[0], to_stop_coords[0]).km
  else:
      return None


# Create a graph representation of bus routes
bus_stop_graph = {}
for index, row in df3.iterrows():
    start = row['Source']
    end = row['Destination']
    distance = row['Distance']

    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance


# Create a dictionary of bus stop coordinates
bus_stop_coordinates = {}
for index, row in df2.iterrows():
    stop = row['stop_id']
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)


# Dijkstra's algorithm for finding shortest path
def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    node_data = {node: {'cost': inf, 'pred': None} for node in graph}
    node_data[src]['cost'] = 0
    visited = set()
    heap = [(0, src)]

    while heap:
        (cost, node) = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph[node].items():
            if neighbor in node_data:
                new_cost = cost + weight
                if new_cost < node_data[neighbor]['cost']:
                    node_data[neighbor]['cost'] = new_cost
                    node_data[neighbor]['pred'] = node
                    heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    shortest_distance, path = node_data[dest]['cost'], reconstruct_path(dest)
    return shortest_distance, path


# Function to calculate bus cost based on distance
def calculate_bus_cost(distance):
    # Example pricing model:
    base_cost = 1500  # Base cost for local buses
    distance_factor = 0.00  # Cost per meter
    cost = base_cost + (distance * distance_factor)
    return cost


# Function to find the nearest bus stops to a given location
def find_nearest_bus_stops(location, bus_stop_coordinates, num_stops=1):
    distances = []
    for stop_id, coords in bus_stop_coordinates.items():
        distance = geodesic(location, coords).km
        distances.append((stop_id, distance))

    distances.sort(key=lambda item: item[1])
    return distances[:num_stops]


# Function to find nearest stops for a route
def find_nearest_stops_for_route(user_location, destination, bus_stop_coordinates, num_stops=1):
    nearest_to_user = find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops)
    nearest_to_destination = find_nearest_bus_stops(destination, bus_stop_coordinates, num_stops)
    return nearest_to_user, nearest_to_destination
# Function to get route options
def get_route_options(user_location, destination, nearest_to_user, nearest_to_destination, bus_stop_graph):
    route_options = []
    for start_stop, _ in nearest_to_user:
        for end_stop, _ in nearest_to_destination:
            try:
                bus_distance, path = dijkstra(bus_stop_graph, float(start_stop), float(end_stop))
                walking_distance_start = geodesic(user_location, bus_stop_coordinates[start_stop]).meters
                walking_distance_end = geodesic(destination, bus_stop_coordinates[end_stop]).meters
                total_distance = bus_distance + walking_distance_start + walking_distance_end
                route_options.append({
                    'start_stop': start_stop,
                    'end_stop': end_stop,
                    'total_distance': total_distance,
                    'path': path
                })
            except KeyError:
                pass  # Handle cases where there's no bus route between the stops

    route_options.sort(key=lambda x: x['total_distance'])
    return route_options[:1]  # Return top 3 options


# Function to display route options on map
def display_route_options_on_map(user_location, destination, route_options, bus_stop_coordinates):
    my_map = folium.Map(location=[(user_location[0] + destination[0]) / 2, (user_location[1] + destination[1]) / 2], zoom_start=13)

    folium.Marker(location=user_location, popup="Your Location", icon=folium.Icon(color='blue')).add_to(my_map)
    folium.Marker(location=destination, popup="Destination", icon=folium.Icon(color='red')).add_to(my_map)

    colors = ['blue', 'orange', 'purple']
    for i, option in enumerate(route_options):
        folium.PolyLine([user_location, bus_stop_coordinates[option['start_stop']]],
                         color='gray', weight=2.5, opacity=1, dash_array='5',
                         tooltip=f"Walk to {option['start_stop']}").add_to(my_map)

        bus_route_coords = [bus_stop_coordinates[stop] for stop in option['path']]
        folium.PolyLine(bus_route_coords, color=colors[i], weight=3.5, opacity=1,
                         tooltip=f"Bus Route: {option['start_stop']} to {option['end_stop']}").add_to(my_map)

        folium.PolyLine([bus_stop_coordinates[option['end_stop']], destination],
                         color='gray', weight=2.5, opacity=1, dash_array='5',
                         tooltip=f"Walk to Destination").add_to(my_map)

    return my_map


# Example usage:
user_location = (11.587485771987394, 104.9168081053572)
destination = (13.284128007937076, 104.06482052949444)

nearest_to_user, nearest_to_destination = find_nearest_stops_for_route(
    user_location, destination, bus_stop_coordinates
)

route_options = get_route_options(
    user_location, destination, nearest_to_user, nearest_to_destination, bus_stop_graph
)

route_map = display_route_options_on_map(user_location, destination, route_options, bus_stop_coordinates)
route_map

In [12]:
import pandas as pd
import sys
from heapq import heappop, heappush
import folium
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Mount Google Drive (if needed)
# drive.mount('/content/drive')

# Load bus stop graph and coordinates data
df1 = pd.read_csv('bus_stop_graph.csv')  # Replace with your path
df2 = pd.read_csv('bus_stop_coordinates.csv')  # Replace with your path

# Create bus_stop_graph and bus_stop_coordinates dictionaries
bus_stop_graph = {}
for index, row in df1.iterrows():
    start = row['Source']
    end = row['Destination']
    distance = row['Distance']
    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance

bus_stop_coordinates = {}
for index, row in df2.iterrows():
    stop = row['stop_id']
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)

# Function to find nearest bus stops to a location
def find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops=3):
    distances = []
    for stop, coords in bus_stop_coordinates.items():
        distance = geodesic(user_location, coords).meters
        distances.append((stop, distance))
    distances.sort(key=lambda x: x[1])  # Sort by distance
    return distances[:num_stops]

# Function to perform Dijkstra's algorithm
def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    # Change to use consistent key type (float in this case)
    node_data = {float(node): {'cost': inf, 'pred': None} for node in graph}  
    # Ensure src is treated as a float key
    node_data[float(src)]['cost'] = 0  
    visited = set()
    heap = [(0, float(src))]  # Ensure src is a float in the heap

    while heap:
        (cost, node) = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph.get(node, {}).items():
            # Ensure neighbor is treated as a float key
            neighbor = float(neighbor)  
            if neighbor not in node_data:
                node_data[neighbor] = {'cost': inf, 'pred': None}
            new_cost = cost + weight
            if new_cost < node_data[neighbor]['cost']:
                node_data[neighbor]['cost'] = new_cost
                node_data[neighbor]['pred'] = node
                heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    # Ensure dest is treated as a float key
    shortest_distance, path = node_data[float(dest)]['cost'], reconstruct_path(float(dest))  
    print("Shortest Distance:", shortest_distance)
    print("Shortest Path:", path)

    return shortest_distance, path

# Get user's location and destination using Geocoding
geolocator = Nominatim(user_agent="bus_app")
user_location_name = input("Enter your location: ")
user_location = geolocator.geocode(user_location_name)
user_location = (user_location.latitude, user_location.longitude)

destination_location_name = input("Enter your destination: ")
destination_location = geolocator.geocode(destination_location_name)
destination_location = (destination_location.latitude, destination_location.longitude)


# Find nearest bus stops to user's location and destination
nearest_source_stops = find_nearest_bus_stops(user_location, bus_stop_coordinates)
nearest_destination_stops = find_nearest_bus_stops(destination_location, bus_stop_coordinates)

print("Nearest source bus stops:")
for stop, distance in nearest_source_stops:
    print(f"- {stop}: {distance:.2f} meters")

print("Nearest destination bus stops:")
for stop, distance in nearest_destination_stops:
    print(f"- {stop}: {distance:.2f} meters")

AttributeError: 'NoneType' object has no attribute 'latitude'

In [ ]:
# Use Dijkstra's algorithm to find the shortest path between the nearest stops
# Assuming you want the path between the first nearest stops
source_stop = float(nearest_source_stops[0][0])  # First nearest source stop ID
destination_stop = float(nearest_destination_stops[0][0])  # First nearest destination stop ID

shortest_distance, path = dijkstra(bus_stop_graph, source_stop, destination_stop)

# Create a map centered around the average coordinates of your stops (adjust as needed)
map_center = [df2['latitude'].mean(), df2['longitude'].mean()]
my_map = folium.Map(location=map_center, zoom_start=8)

# Add markers for start and destination stops (if needed)
if source_stop in bus_stop_coordinates:
    folium.Marker(
        location=bus_stop_coordinates[source_stop],
        popup=f"Start: {source_stop}",
        icon=folium.Icon(color='green')
    ).add_to(my_map)

if destination_stop in bus_stop_coordinates:
    folium.Marker(
        location=bus_stop_coordinates[destination_stop],
        popup=f"Destination: {destination_stop}",
        icon=folium.Icon(color='red')
    ).add_to(my_map)


# Draw lines between connected bus stops
for start_stop, neighbors in bus_stop_graph.items():
  for end_stop, distance in neighbors.items():
    if start_stop in bus_stop_coordinates and end_stop in bus_stop_coordinates:
      folium.PolyLine(
          locations=[bus_stop_coordinates[start_stop], bus_stop_coordinates[end_stop]],
          color='blue',
          weight=1,
          opacity=0.7
      ).add_to(my_map)

# Draw the shortest path on the map using PolyLine (if needed)
if path:
    path_coords = [bus_stop_coordinates[stop_id] for stop_id in path if stop_id in bus_stop_coordinates]
    if path_coords:
        folium.PolyLine(locations=path_coords, color='red', weight=3).add_to(my_map)

# Display the map
my_map

In [ ]:
df1 = pd.read_csv('bus_stop_graph.csv')  # Replace with your path
df2 = pd.read_csv('bus_stop_coordinates.csv')  # Replace with your path

# Create bus_stop_graph and bus_stop_coordinates dictionaries
bus_stop_graph = {}
for index, row in df1.iterrows():
    start = row['Source']
    end = row['Destination']
    distance = row['Distance']
    if start not in bus_stop_graph:
        bus_stop_graph[start] = {}
    bus_stop_graph[start][end] = distance

bus_stop_coordinates = {}
for index, row in df2.iterrows():
    stop = row['stop_id']
    lat = row['latitude']
    lon = row['longitude']
    bus_stop_coordinates[stop] = (lat, lon)

# Function to perform Dijkstra's algorithm
def dijkstra(graph, src, dest, walking_threshold=200):
    inf = sys.maxsize
    # Change to use consistent key type (int in this case)
    node_data = {int(node): {'cost': inf, 'pred': None} for node in graph}
    # Ensure src is treated as an int key
    node_data[int(src)]['cost'] = 0
    visited = set()
    heap = [(0, int(src))]  # Ensure src is an int in the heap

    while heap:
        (cost, node) = heappop(heap)
        if node in visited:
            continue
        visited.add(node)

        for neighbor, weight in graph.get(node, {}).items():
            # Ensure neighbor is treated as an int key
            neighbor = int(neighbor)
            if neighbor not in node_data:
                node_data[neighbor] = {'cost': inf, 'pred': None}
            new_cost = cost + weight
            if new_cost < node_data[neighbor]['cost']:
                node_data[neighbor]['cost'] = new_cost
                node_data[neighbor]['pred'] = node
                heappush(heap, (new_cost, neighbor))

    def reconstruct_path(node):
        path = []
        while node is not None:
            path.append(node)
            node = node_data[node]['pred']
        return path[::-1]

    # Check if the destination is reachable
    if int(dest) in node_data:
        shortest_distance, path = node_data[int(dest)]['cost'], reconstruct_path(int(dest))
        print("Shortest Distance:", shortest_distance)
        print("Shortest Path:", path)
        return shortest_distance, path
    else:
        #print(f"No path found to destination stop {dest}. It's likely unreachable from the source stop.")
        # If no path is found, return a path consisting only of the destination
        return None, [dest]  # Return a list with only the destination

# Get user's location and destination using Geocoding
geolocator = Nominatim(user_agent="bus_app")
user_location_name = input("Enter your location: ")
user_location = geolocator.geocode(user_location_name)
user_location = (user_location.latitude, user_location.longitude)

destination_location_name = input("Enter your destination: ")
destination_location = geolocator.geocode(destination_location_name)
destination_location = (destination_location.latitude, destination_location.longitude)

def find_nearest_bus_stops(user_location, bus_stop_coordinates, num_stops=3):
    """Finds the nearest bus stops to a given location."""
    distances = []
    for stop_id, (lat, lon) in bus_stop_coordinates.items():
        distance = geodesic(user_location, (lat, lon)).meters
        distances.append((stop_id, distance))
    distances.sort(key=lambda x: x[1])  # Sort by distance
    return distances[:num_stops]

# Find nearest bus stops to user's location and destination
nearest_source_stops = find_nearest_bus_stops(user_location, bus_stop_coordinates)
nearest_destination_stops = find_nearest_bus_stops(destination_location, bus_stop_coordinates)

print("Nearest source bus stops:")
for stop, distance in nearest_source_stops:
    print(f"- {stop}: {distance:.2f} meters")

print("Nearest destination bus stops:")
for stop, distance in nearest_destination_stops:
    print(f"- {stop}: {distance:.2f} meters")

In [ ]:
# Use Dijkstra's algorithm to find the shortest path between the nearest stops
if nearest_source_stops and nearest_destination_stops:
    source_stop = float(nearest_source_stops[0][0])  # First nearest source stop ID
    destination_stop = float(nearest_destination_stops[0][0])  # First nearest destination stop ID

    shortest_distance, path = dijkstra(bus_stop_graph, source_stop, destination_stop)

    if shortest_distance is None:
        print(f"No path found to destination stop {destination_stop} from {source_stop}. Trying alternative destinations...")

        alternative_paths = []  # Store alternative paths

        # Function to calculate distance to a location (user or destination)
        def distance_to_location(stop_id, location):
            stop_coords = bus_stop_coordinates.get(stop_id)
            if stop_coords:
                return geodesic(location, stop_coords).meters
            return float('inf')  # If stop coordinates are not found

        # Try alternative destination stops first
        for i in range(1, min(len(nearest_destination_stops), 3)):
            alt_destination_stop = float(nearest_destination_stops[i][0])
            alt_shortest_distance, alt_path = dijkstra(bus_stop_graph, source_stop, alt_destination_stop)

            if alt_shortest_distance is not None:
                # Calculate distance of alternative path's destination to original destination
                dist_to_dest = distance_to_location(alt_destination_stop, destination_location)
                alternative_paths.append((alt_shortest_distance, alt_path, alt_destination_stop, dist_to_dest))

        # If not enough alternatives found, try alternative source stops
        if len(alternative_paths) < 2:
            for i in range(1, min(len(nearest_source_stops), 3)):
                alt_source_stop = float(nearest_source_stops[i][0])
                alt_shortest_distance, alt_path = dijkstra(bus_stop_graph, alt_source_stop, destination_stop)

                if alt_shortest_distance is not None:
                    # Calculate distance of alternative path's source to user location
                    dist_to_user = distance_to_location(alt_source_stop, user_location)
                    alternative_paths.append((alt_shortest_distance, alt_path, destination_stop, dist_to_user))

        # Sort alternative paths by distance to original destination or user location
        alternative_paths.sort(key=lambda x: x[3])

        if alternative_paths:
            print("\nAlternative Paths:")
            for distance, path, dest_stop, _ in alternative_paths[:2]:  # Print top 2
                print(f"  - To stop {dest_stop}: Distance = {distance}, Path = {path}")
        else:
            print("No alternative paths found.")